# 1. SETTINGS

In [ ]:
# libraries
import pandas as pd
import numpy as np
import scipy.stats
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

In [ ]:
# pandas options
pd.set_option("display.max_columns", None)

In [ ]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# garbage collection
import gc
gc.enable()

# 2. FUNCTIONS

In [ ]:
#FUNCTION FOR COUNTING MISSINGS
def count_missings(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum() / data.isnull().count() * 100).sort_values(ascending = False)
    table = pd.concat([total, percent], axis = 1, keys = ["Total", "Percent"])
    table = table[table["Total"] > 0]
    return table

In [ ]:
#FUNCTION FOR CONVERTING DATES
def convert_days(data, features, t = 12, rounding = True, replace = False):
    for var in features:
        if replace == True:
            if rounding == True:
                data[var] = round(-data[var]/t)
            else:
                data[var] = -data[var]/t
            data[var][data[var] < 0] = None
        else:
            if rounding == True:
                data["CONVERTED_" + str(var)] = round(-data[var]/t)
            else:
                data["CONVERTED_" + str(var)] = -data[var]/t
            data["CONVERTED_" + str(var)][data["CONVERTED_" + str(var)] < 0] = None
    return data

In [ ]:
#FUNCTION FOR CREATING LOGARITHMS
def create_logs(data, features, replace = False):
    for var in features:
        if replace == True:
            data[var] = np.log(data[var].abs() + 1)
        else:
            data["LOG_" + str(var)] = np.log(data[var].abs() + 1)
    return data

In [ ]:
#UNCTION FOR CREATING FLAGS FOR MISSINGS
def create_null_flags(data, features = None):
    if features == None:
        features = data.columns
    for var in features:
        num_null = data[var].isnull() + 0
        if num_null.sum() > 0:
            data["ISNULL_" + str(var)] = num_null
    return data

In [ ]:
#UNCTION FOR TREATING FACTORS
def treat_factors(data, method = "label"):

    # label encoding
    if method == "label":
        factors = [f for f in data.columns if data[f].dtype == "object"]
        for var in factors:
            data[var], _ = pd.factorize(data[var])

    # dummy encoding
    if method == "dummy":
        data = pd.get_dummies(data, drop_first = True)

    # dataset
    return data

In [ ]:
##### FUNCTION FOR COMPUTING ACCEPT/REJECT RATIOS
def compute_accept_reject_ratio(data, lags = [1, 3, 5]):

    # preparations
    dec_prev = data[["SK_ID_CURR", "SK_ID_PREV", "DAYS_DECISION", "NAME_CONTRACT_STATUS"]]
    dec_prev["DAYS_DECISION"] = -dec_prev["DAYS_DECISION"]
    dec_prev = dec_prev.sort_values(by = ["SK_ID_CURR", "DAYS_DECISION"])
    dec_prev = pd.get_dummies(dec_prev)

    # compuatation
    for t in lags:

        # acceptance ratios
        tmp = dec_prev[["SK_ID_CURR", "NAME_CONTRACT_STATUS_Approved"]].groupby(["SK_ID_CURR"]).head(1)
        tmp = tmp.groupby(["SK_ID_CURR"], as_index = False).mean()
        tmp.columns = ["SK_ID_CURR", "APPROVE_RATIO_" + str(t)]
        data = data.merge(tmp, how = "left", on = "SK_ID_CURR")

        # rejection ratios
        tmp = dec_prev[["SK_ID_CURR", "NAME_CONTRACT_STATUS_Refused"]].groupby(["SK_ID_CURR"]).head(1)
        tmp = tmp.groupby(["SK_ID_CURR"], as_index = False).mean()
        tmp.columns = ["SK_ID_CURR", "REJECT_RATIO_" + str(t)]
        data = data.merge(tmp, how = "left", on = "SK_ID_CURR")

    # dataset
    return data

In [ ]:
##### FUNCTION FOR AGGREGATING DATA
def aggregate_data(data, id_var, label = None):


    ### SEPARATE FEATURES

    # display info
    print("- Preparing the dataset...")

    # find factors
    data_factors = [f for f in data.columns if data[f].dtype == "object"]

    # partition subsets
    num_data = data[list(set(data.columns) - set(data_factors))]
    fac_data = data[[id_var] + data_factors]

    # display info
    num_facs = fac_data.shape[1] - 1
    num_nums = num_data.shape[1] - 1
    print("- Extracted %.0f factors and %.0f numerics..." % (num_facs, num_nums))


    ##### AGGREGATION

    # aggregate numerics
    if (num_nums > 0):
        print("- Aggregating numeric features...")
        num_data = num_data.groupby(id_var).agg(["mean", "std", "min", "max"])
        num_data.columns = ["_".join(col).strip() for col in num_data.columns.values]
        num_data = num_data.sort_index()

    # aggregate factors
    if (num_facs > 0):
        print("- Aggregating factor features...")
        fac_data = fac_data.groupby(id_var).agg([("mode",   lambda x: scipy.stats.mode(x)[0][0]),
                                                 ("unique", lambda x: x.nunique())])
        fac_data.columns = ["_".join(col).strip() for col in fac_data.columns.values]
        fac_data = fac_data.sort_index()


    ##### MERGER

    # merge numerics and factors
    if ((num_facs > 0) & (num_nums > 0)):
        agg_data = pd.concat([num_data, fac_data], axis = 1)

    # use factors only
    if ((num_facs > 0) & (num_nums == 0)):
        agg_data = fac_data

    # use numerics only
    if ((num_facs == 0) & (num_nums > 0)):
        agg_data = num_data


    ##### LAST STEPS

    # update labels
    if label != None:
        agg_data.columns = [label + "_" + str(col) for col in agg_data.columns]

    # impute zeros for SD
    #stdevs = agg_data.filter(like = "_std").columns
    #for var in stdevs:
    #    agg_data[var].fillna(0, inplace = True)

    # display info
    print("- Final dimensions:", agg_data.shape)

    # return dataset
    return agg_data

# 3. DATA IMPORT

In [ ]:
# import data
application_train_df = pd.read_csv("/content/application_train.csv")
application_test_df  = pd.read_csv("/content/application_test.csv")
bureau_df  = pd.read_csv("/content/bureau.csv")
bureau_balance_df  = pd.read_csv("/content/bureau_balance.csv")
previous_application_df  = pd.read_csv("/content/previous_application.csv")
credit_card_balance_df  = pd.read_csv("/content/credit_card_balance.csv")
pos_cash_balance_df  = pd.read_csv("/content/POS_CASH_balance.csv")
installments_payments_df  = pd.read_csv("/content/installments_payments.csv")

In [ ]:
# check dimensions
print("Application:", application_train_df.shape, application_test_df.shape)
print("Buro:", bureau_df.shape)
print("Bbal:", bureau_balance_df.shape)
print("Prev:", previous_application_df.shape)
print("Card:", credit_card_balance_df.shape)
print("Poca:", pos_cash_balance_df.shape)
print("Inst:", installments_payments_df.shape)

Application: (36912, 122) (48744, 121)
Buro: (1716428, 17)
Bbal: (13875566, 3)
Prev: (772728, 37)
Card: (1702418, 23)
Poca: (7691365, 8)
Inst: (3491814, 8)


In [ ]:
y = application_train_df[["SK_ID_CURR", "TARGET"]]
y["TARGET"] = y["TARGET"].astype(float)
del application_train_df["TARGET"]

# 4. PREPROCESSING

## 4.1. APPLICATION DATA

In [ ]:
# concatenate application data
appl = pd.concat([application_train_df, application_test_df])
del application_train_df, application_test_df

In [ ]:
### FEATURE ENGINEERING

# income ratios
appl["CREDIT_BY_INCOME"]      = appl["AMT_CREDIT"]      / appl["AMT_INCOME_TOTAL"]
appl["ANNUITY_BY_INCOME"]     = appl["AMT_ANNUITY"]     / appl["AMT_INCOME_TOTAL"]
appl["GOODS_PRICE_BY_INCOME"] = appl["AMT_GOODS_PRICE"] / appl["AMT_INCOME_TOTAL"]
appl["INCOME_PER_PERSON"]     = appl["AMT_INCOME_TOTAL"] / appl["CNT_FAM_MEMBERS"]

# career ratio
appl["PERCENT_WORKED"] = appl["DAYS_EMPLOYED"] / appl["DAYS_BIRTH"]
appl["PERCENT_WORKED"][appl["PERCENT_WORKED"] < 0] = None

# number of adults
appl["CNT_ADULTS"] = appl["CNT_FAM_MEMBERS"] - appl["CNT_CHILDREN"]
appl['CHILDREN_RATIO'] = appl['CNT_CHILDREN'] / appl['CNT_FAM_MEMBERS']

# number of overall payments
appl['ANNUITY LENGTH'] = appl['AMT_CREDIT'] / appl['AMT_ANNUITY']

# external sources
#appl["EXT_SOURCE_MIN"]  = appl[["EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3"]].min(axis = 1)
#appl["EXT_SOURCE_MAX"]  = appl[["EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3"]].max(axis = 1)
appl["EXT_SOURCE_MEAN"] = appl[["EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3"]].mean(axis = 1)
#appl["EXT_SOURCE_SD"]   = appl[["EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3"]].std(axis = 1)
appl["NUM_EXT_SOURCES"] = 3 - (appl["EXT_SOURCE_1"].isnull().astype(int) +
                               appl["EXT_SOURCE_2"].isnull().astype(int) +
                               appl["EXT_SOURCE_3"].isnull().astype(int))

# number of documents
doc_vars = ["FLAG_DOCUMENT_2",  "FLAG_DOCUMENT_3",  "FLAG_DOCUMENT_4",  "FLAG_DOCUMENT_5",  "FLAG_DOCUMENT_6",
            "FLAG_DOCUMENT_7",  "FLAG_DOCUMENT_8",  "FLAG_DOCUMENT_9",  "FLAG_DOCUMENT_10", "FLAG_DOCUMENT_11",
            "FLAG_DOCUMENT_12", "FLAG_DOCUMENT_13", "FLAG_DOCUMENT_14", "FLAG_DOCUMENT_15", "FLAG_DOCUMENT_16",
            "FLAG_DOCUMENT_17", "FLAG_DOCUMENT_18", "FLAG_DOCUMENT_19", "FLAG_DOCUMENT_20", "FLAG_DOCUMENT_21"]
appl["NUM_DOCUMENTS"] = appl[doc_vars].sum(axis = 1)

# application date
appl["DAY_APPR_PROCESS_START"] = "Working day"
appl["DAY_APPR_PROCESS_START"][(appl["WEEKDAY_APPR_PROCESS_START"] == "SATURDAY") |
                               (appl["WEEKDAY_APPR_PROCESS_START"] == "SUNDAY")] = "Weekend"

# logarithms
log_vars = ["AMT_CREDIT", "AMT_INCOME_TOTAL", "AMT_GOODS_PRICE", "AMT_ANNUITY"]
appl = create_logs(appl, log_vars, replace = True)

# convert days
day_vars = ["DAYS_BIRTH", "DAYS_REGISTRATION", "DAYS_ID_PUBLISH", "DAYS_EMPLOYED", "DAYS_LAST_PHONE_CHANGE"]
appl = convert_days(appl, day_vars, t = 30, rounding = True, replace = True)

# age ratios
appl["OWN_CAR_AGE_RATIO"] = appl["OWN_CAR_AGE"] / appl["DAYS_BIRTH"]
appl["DAYS_ID_PUBLISHED_RATIO"] = appl["DAYS_ID_PUBLISH"] / appl["DAYS_BIRTH"]
appl["DAYS_REGISTRATION_RATIO"] = appl["DAYS_REGISTRATION"] / appl["DAYS_BIRTH"]
appl["DAYS_LAST_PHONE_CHANGE_RATIO"] = appl["DAYS_LAST_PHONE_CHANGE"] / appl["DAYS_BIRTH"]


##### FEATURE REMOVAL
drops = ['APARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 'COMMONAREA_MEDI', 'ELEVATORS_MEDI', 'ENTRANCES_MEDI',
         'FLOORSMAX_MEDI', 'FLOORSMIN_MEDI', 'LANDAREA_MEDI', 'LIVINGAPARTMENTS_MEDI', 'LIVINGAREA_MEDI',
         'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAREA_MEDI','YEARS_BEGINEXPLUATATION_MEDI', 'YEARS_BUILD_MEDI',
         'APARTMENTS_MODE', 'BASEMENTAREA_MODE', 'COMMONAREA_MODE','ELEVATORS_MODE', 'ENTRANCES_MODE',
         'FLOORSMAX_MODE', 'FLOORSMIN_MODE', 'LANDAREA_MODE', 'LIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE',
         'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE', 'TOTALAREA_MODE',  'YEARS_BEGINEXPLUATATION_MODE']
appl = appl.drop(columns = drops)

In [ ]:
# rename features
appl.columns = ["SK_ID_CURR"] + ["app_" + str(col) for col in appl.columns if col not in "SK_ID_CURR"]

In [ ]:
# check data
appl.head()

,SK_ID_CURR,app_NAME_CONTRACT_TYPE,app_CODE_GENDER,app_FLAG_OWN_CAR,app_FLAG_OWN_REALTY,app_CNT_CHILDREN,app_AMT_INCOME_TOTAL,app_AMT_CREDIT,app_AMT_ANNUITY,app_AMT_GOODS_PRICE,app_NAME_TYPE_SUITE,app_NAME_INCOME_TYPE,app_NAME_EDUCATION_TYPE,app_NAME_FAMILY_STATUS,app_NAME_HOUSING_TYPE,app_REGION_POPULATION_RELATIVE,app_DAYS_BIRTH,app_DAYS_EMPLOYED,app_DAYS_REGISTRATION,app_DAYS_ID_PUBLISH,app_OWN_CAR_AGE,app_FLAG_MOBIL,app_FLAG_EMP_PHONE,app_FLAG_WORK_PHONE,app_FLAG_CONT_MOBILE,app_FLAG_PHONE,app_FLAG_EMAIL,app_OCCUPATION_TYPE,app_CNT_FAM_MEMBERS,app_REGION_RATING_CLIENT,app_REGION_RATING_CLIENT_W_CITY,app_WEEKDAY_APPR_PROCESS_START,app_HOUR_APPR_PROCESS_START,app_REG_REGION_NOT_LIVE_REGION,app_REG_REGION_NOT_WORK_REGION,app_LIVE_REGION_NOT_WORK_REGION,app_REG_CITY_NOT_LIVE_CITY,app_REG_CITY_NOT_WORK_CITY,app_LIVE_CITY_NOT_WORK_CITY,app_ORGANIZATION_TYPE,app_EXT_SOURCE_1,app_EXT_SOURCE_2,app_EXT_SOURCE_3,app_APARTMENTS_AVG,app_BASEMENTAREA_AVG,app_YEARS_BEGINEXPLUATATION_AVG,app_YEARS_BUILD_AVG,app_COMMONAREA_AVG,app_ELEVATORS_AVG,app_ENTRANCES_AVG,app_FLOORSMAX_AVG,app_FLOORSMIN_AVG,app_LANDAREA_AVG,app_LIVINGAPARTMENTS_AVG,app_LIVINGAREA_AVG,app_NONLIVINGAPARTMENTS_AVG,app_NONLIVINGAREA_AVG,app_YEARS_BUILD_MODE,app_FONDKAPREMONT_MODE,app_HOUSETYPE_MODE,app_WALLSMATERIAL_MODE,app_EMERGENCYSTATE_MODE,app_OBS_30_CNT_SOCIAL_CIRCLE,app_DEF_30_CNT_SOCIAL_CIRCLE,app_OBS_60_CNT_SOCIAL_CIRCLE,app_DEF_60_CNT_SOCIAL_CIRCLE,app_DAYS_LAST_PHONE_CHANGE,app_FLAG_DOCUMENT_2,app_FLAG_DOCUMENT_3,app_FLAG_DOCUMENT_4,app_FLAG_DOCUMENT_5,app_FLAG_DOCUMENT_6,app_FLAG_DOCUMENT_7,app_FLAG_DOCUMENT_8,app_FLAG_DOCUMENT_9,app_FLAG_DOCUMENT_10,app_FLAG_DOCUMENT_11,app_FLAG_DOCUMENT_12,app_FLAG_DOCUMENT_13,app_FLAG_DOCUMENT_14,app_FLAG_DOCUMENT_15,app_FLAG_DOCUMENT_16,app_FLAG_DOCUMENT_17,app_FLAG_DOCUMENT_18,app_FLAG_DOCUMENT_19,app_FLAG_DOCUMENT_20,app_FLAG_DOCUMENT_21,app_AMT_REQ_CREDIT_BUREAU_HOUR,app_AMT_REQ_CREDIT_BUREAU_DAY,app_AMT_REQ_CREDIT_BUREAU_WEEK,app_AMT_REQ_CREDIT_BUREAU_MON,app_AMT_REQ_CREDIT_BUREAU_QRT,app_AMT_REQ_CREDIT_BUREAU_YEAR,app_CREDIT_BY_INCOME,app_ANNUITY_BY_INCOME,app_GOODS_PRICE_BY_INCOME,app_INCOME_PER_PERSON,app_PERCENT_WORKED,app_CNT_ADULTS,app_CHILDREN_RATIO,app_ANNUITY LENGTH,app_EXT_SOURCE_MEAN,app_NUM_EXT_SOURCES,app_NUM_DOCUMENTS,app_DAY_APPR_PROCESS_START,app_OWN_CAR_AGE_RATIO,app_DAYS_ID_PUBLISHED_RATIO,app_DAYS_REGISTRATION_RATIO,app_DAYS_LAST_PHONE_CHANGE_RATIO
0,100002,Cash loans,M,N,Y,0,12.218500,12.915581,10.114619,12.768544,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,315.0,21.0,122.0,71.0,NaN,1.0,1.0,0.0,1.0,1.0,0.0,Laborers,1.0,2.0,2.0,WEDNESDAY,10.0,0.0,0.0,0.0,0.0,0.0,0.0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.6341,reg oper account,block of flats,"Stone, brick",No,2.0,2.0,2.0,2.0,38.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.007889,0.121978,1.733333,202500.0,0.067329,1.0,0.0,16.461104,0.161787,3,1.0,Working day,NaN,0.225397,0.387302,0.120635
1,100003,Cash loans,F,N,N,0,12.506181,14.072865,10.482892,13.937287,Family,State servant,Higher education,Married,House / apartment,0.003541,559.0,40.0,40.0,10.0,NaN,1.0,1.0,0.0,1.0,1.0,0.0,Core staff,2.0,1.0,1.0,MONDAY,11.0,0.0,0.0,0.0,0.0,0.0,0.0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.8040,reg oper account,block of flats,Block,No,1.0,0.0,1.0,0.0,28.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.790750,0.132217,4.183333,135000.0,0.070862,2.0,0.0,36.234085,0.466757,2,1.0,Working day,NaN,0.017889,0.071556,0.050089
2,100004,Revolving loans,M,Y,Y,0,11.119898,11.813037,8.817446,11.813037,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,635.0,8.0,142

In [ ]:
# count missings
nas = count_missings(appl)
nas.head()

,Total,Percent
app_COMMONAREA_AVG,59374,69.316802
app_NONLIVINGAPARTMENTS_AVG,59023,68.907023
app_FONDKAPREMONT_MODE,58102,67.831792
app_LIVINGAPARTMENTS_AVG,58095,67.823620
app_FLOORSMIN_AVG,57541,67.176847


## 4.2. CREDIT BUREAU DATA

### 4.2.1. BUREAU BALANCE


In [ ]:
# check bbal data
bureau_balance_df.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


In [ ]:
### FEATURE ENGINEERING

# loan default score
bureau_balance_df["NUM_STATUS"] = 0
bureau_balance_df["NUM_STATUS"][bureau_balance_df["STATUS"] == "X"] = None
bureau_balance_df["NUM_STATUS"][bureau_balance_df["STATUS"] == "1"] = 1
bureau_balance_df["NUM_STATUS"][bureau_balance_df["STATUS"] == "2"] = 2
bureau_balance_df["NUM_STATUS"][bureau_balance_df["STATUS"] == "3"] = 3
bureau_balance_df["NUM_STATUS"][bureau_balance_df["STATUS"] == "4"] = 4
bureau_balance_df["NUM_STATUS"][bureau_balance_df["STATUS"] == "5"] = 5
bureau_balance_df["LOAN_SCORE"] = bureau_balance_df["NUM_STATUS"] / (abs(bureau_balance_df["MONTHS_BALANCE"]) + 1)
loan_score = bureau_balance_df.groupby("SK_ID_BUREAU", as_index = False).LOAN_SCORE.sum()
del bureau_balance_df["NUM_STATUS"]
del bureau_balance_df["LOAN_SCORE"]

# dummy encoding for STATUS
bureau_balance_df = pd.get_dummies(bureau_balance_df, columns = ["STATUS"], prefix = "STATUS")

In [ ]:
# count missings
nas = count_missings(bureau_balance_df)
nas.head()

,Total,Percent


In [ ]:
### AGGREGATIONS

# total month count
cnt_mon = bureau_balance_df[["SK_ID_BUREAU", "MONTHS_BALANCE"]].groupby("SK_ID_BUREAU").count()
del bureau_balance_df["MONTHS_BALANCE"]

# aggregate data
agg_bbal = bureau_balance_df.groupby("SK_ID_BUREAU").mean()

# add total month count
agg_bbal["MONTH_COUNT"] = cnt_mon

# add loan score
agg_bbal = agg_bbal.merge(loan_score, how = "left", on = "SK_ID_BUREAU")

In [ ]:
# count missings
nas = count_missings(agg_bbal)
nas.head()

,Total,Percent


In [ ]:
# check data
agg_bbal.head()

,SK_ID_BUREAU,STATUS_0,STATUS_1,STATUS_2,STATUS_3,STATUS_4,STATUS_5,STATUS_C,STATUS_X,MONTH_COUNT,LOAN_SCORE
0,5001710,0.060241,0.0,0.0,0.0,0.0,0.0,0.578313,0.361446,83,0.0
1,5001711,0.750000,0.0,0.0,0.0,0.0,0.0,0.000000,0.250000,4,0.0
2,5001712,0.526316,0.0,0.0,0.0,0.0,0.0,0.473684,0.000000,19,0.0
3,5001713,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,22,0.0
4,5001714,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,15,0.0


In [ ]:
# clear memory
del bureau_balance_df

### 4.2.2. BUREAU


In [ ]:
# check buro data
bureau_df.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [ ]:
### MERGE
bureau_df = bureau_df.merge(right = agg_bbal.reset_index(), how = "left", on = "SK_ID_BUREAU")

In [ ]:
##### FEATURE ENGINEERING

# number of buro loans
cnt_buro = bureau_df[["SK_ID_CURR", "SK_ID_BUREAU"]].groupby(["SK_ID_CURR"], as_index = False).count()
cnt_buro.columns = ["SK_ID_CURR", "CNT_BURO_LOANS"]
bureau_df = bureau_df.merge(cnt_buro, how = "left", on = "SK_ID_CURR")

# amount ratios
bureau_df["AMT_SUM_OVERDUE_RATIO_1"] = bureau_df["AMT_CREDIT_SUM_OVERDUE"] / bureau_df["AMT_ANNUITY"]
bureau_df["AMT_SUM_OVERDUE_RATIO_2"] = bureau_df["AMT_CREDIT_SUM_OVERDUE"] / bureau_df["AMT_CREDIT_SUM"]
bureau_df["AMT_MAX_OVERDUE_RATIO_1"] = bureau_df["AMT_CREDIT_MAX_OVERDUE"] / bureau_df["AMT_ANNUITY"]
bureau_df["AMT_MAX_OVERDUE_RATIO_2"] = bureau_df["AMT_CREDIT_MAX_OVERDUE"] / bureau_df["AMT_CREDIT_SUM"]
bureau_df["AMT_SUM_DEBT_RATIO_1"]    = bureau_df["AMT_CREDIT_SUM_DEBT"] / bureau_df["AMT_CREDIT_SUM"]
bureau_df["AMT_SUM_DEBT_RATIO_2"]    = bureau_df["AMT_CREDIT_SUM_DEBT"] / bureau_df["AMT_CREDIT_SUM_LIMIT"]

# logarithms
log_vars = ["AMT_CREDIT_SUM", "AMT_CREDIT_SUM_DEBT", "AMT_CREDIT_SUM_LIMIT", "AMT_CREDIT_SUM_OVERDUE", "AMT_ANNUITY"]
bureau_df = create_logs(bureau_df, log_vars, replace = True)

# convert days
day_vars = ["DAYS_CREDIT", "CREDIT_DAY_OVERDUE", "DAYS_CREDIT_ENDDATE", "DAYS_ENDDATE_FACT", "DAYS_CREDIT_UPDATE"]
bureau_df = convert_days(bureau_df, day_vars, t = 1, rounding = False, replace = True)

# recency-weighted loan score
bureau_df["WEIGHTED_LOAN_SCORE"] = bureau_df["LOAN_SCORE"] / (bureau_df["DAYS_CREDIT"] / 12)

# day differences
bureau_df["DAYS_END_DIFF_1"] = bureau_df["DAYS_ENDDATE_FACT"]   - bureau_df["DAYS_CREDIT_ENDDATE"]
bureau_df["DAYS_END_DIFF_2"] = bureau_df["DAYS_CREDIT_UPDATE"]  - bureau_df["DAYS_CREDIT_ENDDATE"]
bureau_df["DAYS_DURATION_1"] = bureau_df["DAYS_CREDIT_ENDDATE"] - bureau_df["DAYS_CREDIT"]
bureau_df["DAYS_DURATION_2"] = bureau_df["DAYS_ENDDATE_FACT"]   - bureau_df["DAYS_CREDIT"]

# number of active buro loans
cnt_buro = bureau_df[["SK_ID_CURR", "CREDIT_ACTIVE"]]
cnt_buro.columns = ["SK_ID_CURR", "CNT_BURO_ACTIVE"]
cnt_buro = cnt_buro[cnt_buro["CNT_BURO_ACTIVE"] == "Active"]
cnt_buro = cnt_buro[["SK_ID_CURR", "CNT_BURO_ACTIVE"]].groupby(["SK_ID_CURR"], as_index = False).count()
bureau_df = bureau_df.merge(cnt_buro, how = "left", on = "SK_ID_CURR")
bureau_df["CNT_BURO_ACTIVE"].fillna(0, inplace = True)

# number of closed buro loans
cnt_buro = bureau_df[["SK_ID_CURR", "CREDIT_ACTIVE"]]
cnt_buro.columns = ["SK_ID_CURR", "CNT_BURO_CLOSED"]
cnt_buro = cnt_buro[cnt_buro["CNT_BURO_CLOSED"] == "Closed"]
cnt_buro = cnt_buro[["SK_ID_CURR", "CNT_BURO_CLOSED"]].groupby(["SK_ID_CURR"], as_index = False).count()
bureau_df = bureau_df.merge(cnt_buro, how = "left", on = "SK_ID_CURR")
bureau_df["CNT_BURO_CLOSED"].fillna(0, inplace = True)

# number of defaulted buro loans
cnt_buro = bureau_df[["SK_ID_CURR", "CREDIT_ACTIVE"]]
cnt_buro.columns = ["SK_ID_CURR", "CNT_BURO_BAD"]
cnt_buro = cnt_buro[cnt_buro["CNT_BURO_BAD"] == "Bad debt"]
cnt_buro = cnt_buro[["SK_ID_CURR", "CNT_BURO_BAD"]].groupby(["SK_ID_CURR"], as_index = False).count()
bureau_df = bureau_df.merge(cnt_buro, how = "left", on = "SK_ID_CURR")
bureau_df["CNT_BURO_BAD"].fillna(0, inplace = True)

In [ ]:
# dummy encodnig for factors
bureau_df = pd.get_dummies(bureau_df, drop_first = True)

In [ ]:
# count missings
nas = count_missings(bureau_df)
nas.head()

,Total,Percent
AMT_MAX_OVERDUE_RATIO_1,1629591,94.940831
AMT_SUM_OVERDUE_RATIO_1,1483326,86.419355
AMT_SUM_DEBT_RATIO_2,1336100,77.841890
WEIGHTED_LOAN_SCORE,1319607,76.880999
STATUS_1,1319606,76.880941


In [ ]:
### AGGREGATIONS

# count previous buro loans
cnt_buro = bureau_df[["SK_ID_CURR", "SK_ID_BUREAU"]].groupby("SK_ID_CURR").count()
del bureau_df["SK_ID_BUREAU"]

# aggregate data
agg_buro = aggregate_data(bureau_df, id_var = "SK_ID_CURR", label = "buro")

# add buro loan count
agg_buro["buro_BURO_COUNT"] = cnt_buro

# clean up
omits = ["WEIGHTED_LOAN_SCORE"]
for var in omits:
    del agg_buro["buro_" + str(var) + "_std"]
    del agg_buro["buro_" + str(var) + "_min"]
    del agg_buro["buro_" + str(var) + "_max"]

- Preparing the dataset...
- Extracted 0 factors and 58 numerics...
- Aggregating numeric features...
- Final dimensions: (305811, 232)


In [ ]:
# count missings
nas = count_missings(agg_buro)
nas.head()

,Total,Percent
buro_AMT_SUM_DEBT_RATIO_2_std,302007,98.756094
buro_AMT_MAX_OVERDUE_RATIO_1_std,288362,94.294188
buro_AMT_MAX_OVERDUE_RATIO_1_max,253792,82.989821
buro_AMT_MAX_OVERDUE_RATIO_1_mean,253792,82.989821
buro_AMT_MAX_OVERDUE_RATIO_1_min,253792,82.989821


In [ ]:
# check data
agg_buro.head()

,buro_CREDIT_ACTIVE_Bad debt_mean,buro_CREDIT_ACTIVE_Bad debt_std,buro_CREDIT_ACTIVE_Bad debt_min,buro_CREDIT_ACTIVE_Bad debt_max,buro_CREDIT_TYPE_Real estate loan_mean,buro_CREDIT_TYPE_Real estate loan_std,buro_CREDIT_TYPE_Real estate loan_min,buro_CREDIT_TYPE_Real estate loan_max,buro_CREDIT_TYPE_Interbank credit_mean,buro_CREDIT_TYPE_Interbank credit_std,buro_CREDIT_TYPE_Interbank credit_min,buro_CREDIT_TYPE_Interbank credit_max,buro_CREDIT_TYPE_Mortgage_mean,buro_CREDIT_TYPE_Mortgage_std,buro_CREDIT_TYPE_Mortgage_min,buro_CREDIT_TYPE_Mortgage_max,buro_STATUS_1_mean,buro_STATUS_1_std,buro_STATUS_1_min,buro_STATUS_1_max,buro_CREDIT_TYPE_Consumer credit_mean,buro_CREDIT_TYPE_Consumer credit_std,buro_CREDIT_TYPE_Consumer credit_min,buro_CREDIT_TYPE_Consumer credit_max,buro_AMT_SUM_DEBT_RATIO_1_mean,buro_AMT_SUM_DEBT_RATIO_1_std,buro_AMT_SUM_DEBT_RATIO_1_min,buro_AMT_SUM_DEBT_RATIO_1_max,buro_CNT_BURO_CLOSED_mean,buro_CNT_BURO_CLOSED_std,buro_CNT_BURO_CLOSED_min,buro_CNT_BURO_CLOSED_max,buro_CREDIT_TYPE_Loan for the purchase of equipment_mean,buro_CREDIT_TYPE_Loan for the purchase of equipment_std,buro_CREDIT_TYPE_Loan for the purchase of equipment_min,buro_CREDIT_TYPE_Loan for the purchase of equipment_max,buro_DAYS_END_DIFF_1_mean,buro_DAYS_END_DIFF_1_std,buro_DAYS_END_DIFF_1_min,buro_DAYS_END_DIFF_1_max,buro_AMT_ANNUITY_mean,buro_AMT_ANNUITY_std,buro_AMT_ANNUITY_min,buro_AMT_ANNUITY_max,buro_MONTH_COUNT_mean,buro_MONTH_COUNT_std,buro_MONTH_COUNT_min,buro_MONTH_COUNT_max,buro_STATUS_2_mean,buro_STATUS_2_std,buro_STATUS_2_min,buro_STATUS_2_max,buro_DAYS_DURATION_1_mean,buro_DAYS_DURATION_1_std,buro_DAYS_DURATION_1_min,buro_DAYS_DURATION_1_max,buro_AMT_MAX_OVERDUE_RATIO_1_mean,buro_AMT_MAX_OVERDUE_RATIO_1_std,buro_AMT_MAX_OVERDUE_RATIO_1_min,buro_AMT_MAX_OVERDUE_RATIO_1_max,buro_CREDIT_CURRENCY_currency 4_mean,buro_CREDIT_CURRENCY_currency 4_std,buro_CREDIT_CURRENCY_currency 4_min,buro_CREDIT_CURRENCY_currency 4_max,buro_STATUS_3_mean,buro_STATUS_3_std,buro_STATUS_3_min,buro_STATUS_3_max,buro_CNT_CREDIT_PROLONG_mean,buro_CNT_CREDIT_PROLONG_std,buro_CNT_CREDIT_PROLONG_min,buro_CNT_CREDIT_PROLONG_max,buro_AMT_CREDIT_SUM_mean,buro_AMT_CREDIT_SUM_std,buro_AMT_CREDIT_SUM_min,buro_AMT_CREDIT_SUM_max,buro_AMT_SUM_OVERDUE_RATIO_1_mean,buro_AMT_SUM_OVERDUE_RATIO_1_std,buro_AMT_SUM_OVERDUE_RATIO_1_min,buro_AMT_SUM_OVERDUE_RATIO_1_max,buro_CREDIT_TYPE_Cash loan (non-earmarked)_mean,buro_CREDIT_TYPE_Cash loan (non-earmarked)_std,buro_CREDIT_TYPE_Cash loan (non-earmarked)_min,buro_CREDIT_TYPE_Cash loan (non-earmarked)_max,buro_CNT_BURO_LOANS_mean,buro_CNT_BURO_LOANS_std,buro_CNT_BURO_LOANS_min,buro_CNT_BURO_LOANS_max,buro_CREDIT_TYPE_Unknown type of loan_mean,buro_CREDIT_TYPE_Unknown type of loan_std,buro_CREDIT_TYPE_Unknown type of loan_min,buro_CREDIT_TYPE_Unknown type of loan_max,buro_STATUS_0_mean,buro_STATUS_0_std,buro_STATUS_0_min,buro_STATUS_0_max,buro_index_mean,buro_index_std,buro_index_min,buro_index_max,buro_CREDIT_DAY_OVERDUE_mean,buro_CREDIT_DAY_OVERDUE_std,buro_CREDIT_DAY_OVERDUE_min,buro_CREDIT_DAY_OVERDUE_max,buro_LOAN_SCORE_mean,buro_LOAN_SCORE_std,buro_LOAN_SCORE_min,buro_LOAN_SCORE_max,buro_DAYS_END_DIFF_2_mean,buro_DAYS_END_DIFF_2_std,buro_DAYS_END_DIFF_2_min,buro_DAYS_END_DIFF_2_max,buro_DAYS_ENDDATE_FACT_mean,buro_DAYS_ENDDATE_FACT_std,buro_DAYS_ENDDATE_FACT_min,buro_DAYS_ENDDATE_FACT_max,buro_CREDIT_TYPE_Credit card_mean,buro_CREDIT_TYPE_Credit card_std,buro_CREDIT_TYPE_Credit card_min,buro_CREDIT_TYPE_Credit card_max,buro_CREDIT_ACTIVE_Sold_mean,buro_CREDIT_ACTIVE_Sold_std,buro_CREDIT_ACTIVE_Sold_min,buro_CREDIT_ACTIVE_Sold_max,buro_CREDIT_CURRENCY_currency 2_mean,buro_CREDIT_CURRENCY_currency 2_std,buro_CREDIT_CURRENCY_currency 2_min,buro_CREDIT_CURRENCY_currency 2_max,buro_CREDIT_TYPE_Mobile operator loan_mean,buro_CREDIT_TYPE_Mobile operator loan_std,buro_CREDIT_TYPE_Mobile operator loan_min,buro_CREDIT_TYPE_Mobile operator loan_max,buro_WEIGHTED_LOAN_SCORE_mean,buro_DAYS_DURATION_2_mean,buro_DAYS_DURATION_2_std,b

In [ ]:
# clear memory
del bureau_df

## 4.3. PREVIOUS LOAN DATA

### 4.3.1. INSTALLMENT PAYMENTS DATA

In [ ]:
# check inst data
installments_payments_df.head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585


In [ ]:
### FEATURE ENGINEERING

# days past due and days before due (no negative values)
installments_payments_df['DPD'] = installments_payments_df['DAYS_ENTRY_PAYMENT'] - installments_payments_df['DAYS_INSTALMENT']
installments_payments_df['DBD'] = installments_payments_df['DAYS_INSTALMENT'] - installments_payments_df['DAYS_ENTRY_PAYMENT']
installments_payments_df['DPD'] = installments_payments_df['DPD'].apply(lambda x: x if x > 0 else 0)
installments_payments_df['DBD'] = installments_payments_df['DBD'].apply(lambda x: x if x > 0 else 0)

# percentage and difference paid in each installment
installments_payments_df['PAYMENT_PERC'] = installments_payments_df['AMT_PAYMENT'] / installments_payments_df['AMT_INSTALMENT']
installments_payments_df['PAYMENT_DIFF'] = installments_payments_df['AMT_INSTALMENT'] - installments_payments_df['AMT_PAYMENT']

# logarithms
log_vars = ["AMT_INSTALMENT", "AMT_PAYMENT"]
installments_payments_df = create_logs(installments_payments_df, log_vars, replace = True)

In [ ]:
# dummy encodnig for factors
installments_payments_df = pd.get_dummies(installments_payments_df, drop_first = True)

In [ ]:
# count missings
nas = count_missings(installments_payments_df)
nas.head()

,Total,Percent
AMT_PAYMENT,1,0.000029
AMT_INSTALMENT,1,0.000029
DAYS_ENTRY_PAYMENT,1,0.000029
DAYS_INSTALMENT,1,0.000029
PAYMENT_DIFF,1,0.000029


In [ ]:
### AGGREGATIONS

# count instalments
cnt_inst = installments_payments_df[["SK_ID_PREV", "NUM_INSTALMENT_NUMBER"]].groupby("SK_ID_PREV").count()
del installments_payments_df["NUM_INSTALMENT_NUMBER"]

# delete ID_CURR
inst_id = installments_payments_df[["SK_ID_CURR", "SK_ID_PREV"]]
del installments_payments_df["SK_ID_CURR"]

# aggregate data
agg_inst = aggregate_data(installments_payments_df, id_var = "SK_ID_PREV")

# add instalment count
agg_inst["inst_INST_COUNT"] = cnt_inst

# put back ID_CURR
inst_id = inst_id.drop_duplicates()
agg_inst = inst_id.merge(right = agg_inst.reset_index(), how = "right", on = "SK_ID_PREV")
del agg_inst["SK_ID_PREV"]

# aggregate data (round 2)
agg_inst = aggregate_data(agg_inst, id_var = "SK_ID_CURR", label = "inst")

- Preparing the dataset...
- Extracted 0 factors and 9 numerics...
- Aggregating numeric features...
- Final dimensions: (278599, 36)
- Preparing the dataset...
- Extracted 0 factors and 37 numerics...
- Aggregating numeric features...
- Final dimensions: (95317, 148)


In [ ]:
# count missings
nas = count_missings(agg_inst)
nas.head()

,Total,Percent
inst_PAYMENT_PERC_std_std,28342,29.734465
inst_AMT_PAYMENT_std_std,28341,29.733416
inst_DPD_std_std,28341,29.733416
inst_NUM_INSTALMENT_VERSION_std_std,28341,29.733416
inst_PAYMENT_DIFF_std_std,28341,29.733416


In [ ]:
# check data
agg_inst.head()

,inst_AMT_PAYMENT_std_mean,inst_AMT_PAYMENT_std_std,inst_AMT_PAYMENT_std_min,inst_AMT_PAYMENT_std_max,inst_DBD_min_mean,inst_DBD_min_std,inst_DBD_min_min,inst_DBD_min_max,inst_DPD_mean_mean,inst_DPD_mean_std,inst_DPD_mean_min,inst_DPD_mean_max,inst_NUM_INSTALMENT_VERSION_std_mean,inst_NUM_INSTALMENT_VERSION_std_std,inst_NUM_INSTALMENT_VERSION_std_min,inst_NUM_INSTALMENT_VERSION_std_max,inst_NUM_INSTALMENT_VERSION_max_mean,inst_NUM_INSTALMENT_VERSION_max_std,inst_NUM_INSTALMENT_VERSION_max_min,inst_NUM_INSTALMENT_VERSION_max_max,inst_DBD_max_mean,inst_DBD_max_std,inst_DBD_max_min,inst_DBD_max_max,inst_NUM_INSTALMENT_VERSION_mean_mean,inst_NUM_INSTALMENT_VERSION_mean_std,inst_NUM_INSTALMENT_VERSION_mean_min,inst_NUM_INSTALMENT_VERSION_mean_max,inst_DAYS_ENTRY_PAYMENT_min_mean,inst_DAYS_ENTRY_PAYMENT_min_std,inst_DAYS_ENTRY_PAYMENT_min_min,inst_DAYS_ENTRY_PAYMENT_min_max,inst_PAYMENT_DIFF_min_mean,inst_PAYMENT_DIFF_min_std,inst_PAYMENT_DIFF_min_min,inst_PAYMENT_DIFF_min_max,inst_DPD_std_mean,inst_DPD_std_std,inst_DPD_std_min,inst_DPD_std_max,inst_PAYMENT_DIFF_max_mean,inst_PAYMENT_DIFF_max_std,inst_PAYMENT_DIFF_max_min,inst_PAYMENT_DIFF_max_max,inst_PAYMENT_PERC_max_mean,inst_PAYMENT_PERC_max_std,inst_PAYMENT_PERC_max_min,inst_PAYMENT_PERC_max_max,inst_DAYS_ENTRY_PAYMENT_mean_mean,inst_DAYS_ENTRY_PAYMENT_mean_std,inst_DAYS_ENTRY_PAYMENT_mean_min,inst_DAYS_ENTRY_PAYMENT_mean_max,inst_PAYMENT_DIFF_mean_mean,inst_PAYMENT_DIFF_mean_std,inst_PAYMENT_DIFF_mean_min,inst_PAYMENT_DIFF_mean_max,inst_DBD_std_mean,inst_DBD_std_std,inst_DBD_std_min,inst_DBD_std_max,inst_DAYS_ENTRY_PAYMENT_max_mean,inst_DAYS_ENTRY_PAYMENT_max_std,inst_DAYS_ENTRY_PAYMENT_max_min,inst_DAYS_ENTRY_PAYMENT_max_max,inst_AMT_INSTALMENT_std_mean,inst_AMT_INSTALMENT_std_std,inst_AMT_INSTALMENT_std_min,inst_AMT_INSTALMENT_std_max,inst_PAYMENT_PERC_mean_mean,inst_PAYMENT_PERC_mean_std,inst_PAYMENT_PERC_mean_min,inst_PAYMENT_PERC_mean_max,inst_DAYS_INSTALMENT_min_mean,inst_DAYS_INSTALMENT_min_std,inst_DAYS_INSTALMENT_min_min,inst_DAYS_INSTALMENT_min_max,inst_PAYMENT_DIFF_std_mean,inst_PAYMENT_DIFF_std_std,inst_PAYMENT_DIFF_std_min,inst_PAYMENT_DIFF_std_max,inst_AMT_PAYMENT_max_mean,inst_AMT_PAYMENT_max_std,inst_AMT_PAYMENT_max_min,inst_AMT_PAYMENT_max_max,inst_DBD_mean_mean,inst_DBD_mean_std,inst_DBD_mean_min,inst_DBD_mean_max,inst_AMT_INSTALMENT_mean_mean,inst_AMT_INSTALMENT_mean_std,inst_AMT_INSTALMENT_mean_min,inst_AMT_INSTALMENT_mean_max,inst_PAYMENT_PERC_std_mean,inst_PAYMENT_PERC_std_std,inst_PAYMENT_PERC_std_min,inst_PAYMENT_PERC_std_max,inst_AMT_INSTALMENT_min_mean,inst_AMT_INSTALMENT_min_std,inst_AMT_INSTALMENT_min_min,inst_AMT_INSTALMENT_min_max,inst_DAYS_INSTALMENT_max_mean,inst_DAYS_INSTALMENT_max_std,inst_DAYS_INSTALMENT_max_min,inst_DAYS_INSTALMENT_max_max,inst_AMT_PAYMENT_min_mean,inst_AMT_PAYMENT_min_std,inst_AMT_PAYMENT_min_min,inst_AMT_PAYMENT_min_max,inst_DAYS_ENTRY_PAYMENT_std_mean,inst_DAYS_ENTRY_PAYMENT_std_std,inst_DAYS_ENTRY_PAYMENT_std_min,inst_DAYS_ENTRY_PAYMENT_std_max,inst_AMT_INSTALMENT_max_mean,inst_AMT_INSTALMENT_max_std,inst_AMT_INSTALMENT_max_min,inst_AMT_INSTALMENT_max_max,inst_DPD_max_mean,inst_DPD_max_std,inst_DPD_max_min,inst_DPD_max_max,inst_DAYS_INSTALMENT_mean_mean,inst_DAYS_INSTALMENT_mean_std,inst_DAYS_INSTALMENT_mean_min,inst_DAYS_INSTALMENT_mean_max,inst_inst_INST_COUNT_mean,inst_inst_INST_COUNT_std,inst_inst_INST_COUNT_min,inst_inst_INST_COUNT_max,inst_NUM_INSTALMENT_VERSION_min_mean,inst_NUM_INSTALMENT_VERSION_min_std,inst_NUM_INSTALMENT_VERSION_min_min,inst_NUM_INSTALMENT_VERSION_min_max,inst_DAYS_INSTALMENT_std_mean,inst_DAYS_INSTALMENT_std_std,inst_DAYS_INSTALMENT_std_min,inst_DAYS_INSTALMENT_std_max,inst_PAYMENT_PERC_min_mean,inst_PAYMENT_PERC_min_std,inst_PAYMENT_PERC_min_min,inst_PAYMENT_PERC_min_max,inst_AMT_PAYMENT_mean_mean,inst_AMT_PAYMENT_mean_std,inst_AMT_PAYMENT_mean_min,inst_AMT_PAYMENT_mean_max,inst_DPD_min_mean,inst_DPD_min_std,inst_DPD_min_min,inst_DPD_min_max
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

In [ ]:
# clear memory
del installments_payments_df

### 4.3.2. POS CASH BALANCE

In [ ]:
# check poca data
pos_cash_balance_df.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0.0,0.0
1,1715348,367990,-33,36.0,35.0,Active,0.0,0.0
2,1784872,397406,-32,12.0,9.0,Active,0.0,0.0
3,1903291,269225,-35,48.0,42.0,Active,0.0,0.0
4,2341044,334279,-35,36.0,35.0,Active,0.0,0.0


In [ ]:
### FEATURE ENGINEERING

# installments percentage
pos_cash_balance_df["INSTALLMENTS_PERCENT"] = pos_cash_balance_df["CNT_INSTALMENT_FUTURE"] / pos_cash_balance_df["CNT_INSTALMENT"]

In [ ]:
# dummy encodnig for factors
pos_cash_balance_df = pd.get_dummies(pos_cash_balance_df, drop_first = True)

In [ ]:
# count missings
nas = count_missings(pos_cash_balance_df)
nas.head()

,Total,Percent
INSTALLMENTS_PERCENT,16639,0.216334
CNT_INSTALMENT_FUTURE,16570,0.215436
CNT_INSTALMENT,16567,0.215397
SK_DPD,1,0.000013
SK_DPD_DEF,1,0.000013


In [ ]:
### AGGREGATIONS

# count months
cnt_mon = pos_cash_balance_df[["SK_ID_PREV", "MONTHS_BALANCE"]].groupby("SK_ID_PREV").count()
del pos_cash_balance_df["MONTHS_BALANCE"]

# delete ID_CURR
poca_id = pos_cash_balance_df[["SK_ID_CURR", "SK_ID_PREV"]]
del pos_cash_balance_df["SK_ID_CURR"]

# aggregate data
agg_poca = aggregate_data(pos_cash_balance_df, id_var = "SK_ID_PREV")

# add month count
agg_poca["poca_MON_COUNT"] = cnt_mon

# put back ID_CURR
poca_id = poca_id.drop_duplicates()
agg_poca = poca_id.merge(right = agg_poca.reset_index(), how = "right", on = "SK_ID_PREV")
del agg_poca["SK_ID_PREV"]

# aggregate data (round 2)
agg_poca = aggregate_data(agg_poca, id_var = "SK_ID_CURR", label = "poca")

- Preparing the dataset...
- Extracted 0 factors and 13 numerics...
- Aggregating numeric features...
- Final dimensions: (926255, 52)
- Preparing the dataset...
- Extracted 0 factors and 53 numerics...
- Aggregating numeric features...
- Final dimensions: (336672, 212)


In [ ]:
# count missings
nas = count_missings(agg_poca)
nas.head()

,Total,Percent
poca_CNT_INSTALMENT_std_std,111392,33.086209
poca_CNT_INSTALMENT_FUTURE_std_std,111392,33.086209
poca_INSTALLMENTS_PERCENT_std_std,111392,33.086209
poca_NAME_CONTRACT_STATUS_Completed_std_std,111127,33.007497
poca_SK_DPD_std_std,111127,33.007497


In [ ]:
# check data
agg_poca.head()

,poca_NAME_CONTRACT_STATUS_Canceled_mean_mean,poca_NAME_CONTRACT_STATUS_Canceled_mean_std,poca_NAME_CONTRACT_STATUS_Canceled_mean_min,poca_NAME_CONTRACT_STATUS_Canceled_mean_max,poca_NAME_CONTRACT_STATUS_Demand_max_mean,poca_NAME_CONTRACT_STATUS_Demand_max_std,poca_NAME_CONTRACT_STATUS_Demand_max_min,poca_NAME_CONTRACT_STATUS_Demand_max_max,poca_NAME_CONTRACT_STATUS_Amortized debt_mean_mean,poca_NAME_CONTRACT_STATUS_Amortized debt_mean_std,poca_NAME_CONTRACT_STATUS_Amortized debt_mean_min,poca_NAME_CONTRACT_STATUS_Amortized debt_mean_max,poca_NAME_CONTRACT_STATUS_Completed_std_mean,poca_NAME_CONTRACT_STATUS_Completed_std_std,poca_NAME_CONTRACT_STATUS_Completed_std_min,poca_NAME_CONTRACT_STATUS_Completed_std_max,poca_NAME_CONTRACT_STATUS_Returned to the store_std_mean,poca_NAME_CONTRACT_STATUS_Returned to the store_std_std,poca_NAME_CONTRACT_STATUS_Returned to the store_std_min,poca_NAME_CONTRACT_STATUS_Returned to the store_std_max,poca_CNT_INSTALMENT_FUTURE_std_mean,poca_CNT_INSTALMENT_FUTURE_std_std,poca_CNT_INSTALMENT_FUTURE_std_min,poca_CNT_INSTALMENT_FUTURE_std_max,poca_CNT_INSTALMENT_FUTURE_max_mean,poca_CNT_INSTALMENT_FUTURE_max_std,poca_CNT_INSTALMENT_FUTURE_max_min,poca_CNT_INSTALMENT_FUTURE_max_max,poca_SK_DPD_DEF_max_mean,poca_SK_DPD_DEF_max_std,poca_SK_DPD_DEF_max_min,poca_SK_DPD_DEF_max_max,poca_CNT_INSTALMENT_std_mean,poca_CNT_INSTALMENT_std_std,poca_CNT_INSTALMENT_std_min,poca_CNT_INSTALMENT_std_max,poca_NAME_CONTRACT_STATUS_XNA_max_mean,poca_NAME_CONTRACT_STATUS_XNA_max_std,poca_NAME_CONTRACT_STATUS_XNA_max_min,poca_NAME_CONTRACT_STATUS_XNA_max_max,poca_SK_DPD_std_mean,poca_SK_DPD_std_std,poca_SK_DPD_std_min,poca_SK_DPD_std_max,poca_NAME_CONTRACT_STATUS_Canceled_std_mean,poca_NAME_CONTRACT_STATUS_Canceled_std_std,poca_NAME_CONTRACT_STATUS_Canceled_std_min,poca_NAME_CONTRACT_STATUS_Canceled_std_max,poca_NAME_CONTRACT_STATUS_Demand_std_mean,poca_NAME_CONTRACT_STATUS_Demand_std_std,poca_NAME_CONTRACT_STATUS_Demand_std_min,poca_NAME_CONTRACT_STATUS_Demand_std_max,poca_NAME_CONTRACT_STATUS_Signed_max_mean,poca_NAME_CONTRACT_STATUS_Signed_max_std,poca_NAME_CONTRACT_STATUS_Signed_max_min,poca_NAME_CONTRACT_STATUS_Signed_max_max,poca_CNT_INSTALMENT_min_mean,poca_CNT_INSTALMENT_min_std,poca_CNT_INSTALMENT_min_min,poca_CNT_INSTALMENT_min_max,poca_CNT_INSTALMENT_mean_mean,poca_CNT_INSTALMENT_mean_std,poca_CNT_INSTALMENT_mean_min,poca_CNT_INSTALMENT_mean_max,poca_NAME_CONTRACT_STATUS_Signed_mean_mean,poca_NAME_CONTRACT_STATUS_Signed_mean_std,poca_NAME_CONTRACT_STATUS_Signed_mean_min,poca_NAME_CONTRACT_STATUS_Signed_mean_max,poca_NAME_CONTRACT_STATUS_Completed_max_mean,poca_NAME_CONTRACT_STATUS_Completed_max_std,poca_NAME_CONTRACT_STATUS_Completed_max_min,poca_NAME_CONTRACT_STATUS_Completed_max_max,poca_NAME_CONTRACT_STATUS_XNA_mean_mean,poca_NAME_CONTRACT_STATUS_XNA_mean_std,poca_NAME_CONTRACT_STATUS_XNA_mean_min,poca_NAME_CONTRACT_STATUS_XNA_mean_max,poca_NAME_CONTRACT_STATUS_Returned to the store_min_mean,poca_NAME_CONTRACT_STATUS_Returned to the store_min_std,poca_NAME_CONTRACT_STATUS_Returned to the store_min_min,poca_NAME_CONTRACT_STATUS_Returned to the store_min_max,poca_NAME_CONTRACT_STATUS_Amortized debt_min_mean,poca_NAME_CONTRACT_STATUS_Amortized debt_min_std,poca_NAME_CONTRACT_STATUS_Amortized debt_min_min,poca_NAME_CONTRACT_STATUS_Amortized debt_min_max,poca_INSTALLMENTS_PERCENT_mean_mean,poca_INSTALLMENTS_PERCENT_mean_std,poca_INSTALLMENTS_PERCENT_mean_min,poca_INSTALLMENTS_PERCENT_mean_max,poca_NAME_CONTRACT_STATUS_Signed_std_mean,poca_NAME_CONTRACT_STATUS_Signed_std_std,poca_NAME_CONTRACT_STATUS_Signed_std_min,poca_NAME_CONTRACT_STATUS_Signed_std_max,poca_NAME_CONTRACT_STATUS_Amortized debt_max_mean,poca_NAME_CONTRACT_STATUS_Amortized debt_max_std,poca_NAME_CONTRACT_STATUS_Amortized debt_max_min,poca_NAME_CONTRACT_STATUS_Amortized debt_max_max,poca_SK_DPD_DEF_min_mean,poca_SK_DPD_DEF_min_std,poca_SK_DPD_DEF_min_min,poca_SK_DPD_DEF_min_max,poca_NAME_CONTRACT_STATUS_Demand_min_mean,poca_NAME_CONTRACT_STATUS_De

In [ ]:
# clear memory
del pos_cash_balance_df

### 4.3.3. CREDIT CARD BALANCE

In [ ]:
# check card data
credit_card_balance_df.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,1800.0,1800.0,0.000,0.000,0.000,0.0,1.0,0.0,1.0,35.0,Active,0.0,0.0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,2250.0,2250.0,60175.080,64875.555,64875.555,1.0,1.0,0.0,0.0,69.0,Active,0.0,0.0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,2250.0,2250.0,26926.425,31460.085,31460.085,0.0,0.0,0.0,0.0,30.0,Active,0.0,0.0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,11925.0,11925.0,224949.285,233048.970,233048.970,1.0,1.0,0.0,0.0,10.0,Active,0.0,0.0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,27000.0,27000.0,443044.395,453919.455,453919.455,0.0,1.0,0.0,1.0,101.0,Active,0.0,0.0


In [ ]:
### FEATURE ENGINEERING

# logarithms
log_vars = ["AMT_BALANCE", "AMT_CREDIT_LIMIT_ACTUAL", "AMT_DRAWINGS_ATM_CURRENT", "AMT_DRAWINGS_CURRENT",
            "AMT_DRAWINGS_OTHER_CURRENT", "AMT_DRAWINGS_POS_CURRENT", "AMT_INST_MIN_REGULARITY",
            "AMT_PAYMENT_CURRENT", "AMT_PAYMENT_TOTAL_CURRENT", "AMT_RECEIVABLE_PRINCIPAL",
            "AMT_RECIVABLE", "AMT_TOTAL_RECEIVABLE"]
credit_card_balance_df = create_logs(credit_card_balance_df, log_vars, replace = True)

In [ ]:
# dummy encodnig for factors
credit_card_balance_df = pd.get_dummies(credit_card_balance_df, drop_first = True)

In [ ]:
# count missings
nas = count_missings(credit_card_balance_df)
nas.head()

,Total,Percent
AMT_PAYMENT_CURRENT,349759,20.544837
CNT_DRAWINGS_POS_CURRENT,344679,20.246438
CNT_DRAWINGS_OTHER_CURRENT,344679,20.246438
AMT_DRAWINGS_OTHER_CURRENT,344678,20.246379
AMT_DRAWINGS_ATM_CURRENT,344678,20.246379


In [ ]:
### AGGREGATIONS

# count months
cnt_mon = credit_card_balance_df[["SK_ID_PREV", "MONTHS_BALANCE"]].groupby("SK_ID_PREV").count()
del credit_card_balance_df["MONTHS_BALANCE"]

# delete ID_CURR
card_id = credit_card_balance_df[["SK_ID_CURR", "SK_ID_PREV"]]
del credit_card_balance_df["SK_ID_CURR"]

# aggregate data
agg_card = aggregate_data(credit_card_balance_df, id_var = "SK_ID_PREV")

# add month count
agg_card["card_MON_COUNT"] = cnt_mon

# put back ID_CURR
card_id = card_id.drop_duplicates()
agg_card = card_id.merge(right = agg_card.reset_index(), how = "right", on = "SK_ID_PREV")
del agg_card["SK_ID_PREV"]

# aggregate data (round 2)
agg_card = aggregate_data(agg_card, id_var = "SK_ID_CURR", label = "card")

- Preparing the dataset...
- Extracted 0 factors and 25 numerics...
- Aggregating numeric features...
- Final dimensions: (102915, 100)
- Preparing the dataset...
- Extracted 0 factors and 101 numerics...
- Aggregating numeric features...
- Final dimensions: (102197, 404)


In [ ]:
# count missings
nas = count_missings(agg_card)
nas.head()

,Total,Percent
card_CNT_DRAWINGS_ATM_CURRENT_std_std,101915,99.724062
card_AMT_DRAWINGS_ATM_CURRENT_std_std,101915,99.724062
card_CNT_DRAWINGS_POS_CURRENT_std_std,101915,99.724062
card_CNT_DRAWINGS_OTHER_CURRENT_std_std,101915,99.724062
card_AMT_DRAWINGS_OTHER_CURRENT_std_std,101915,99.724062


In [ ]:
# check data
agg_card.head()

,card_AMT_RECIVABLE_min_mean,card_AMT_RECIVABLE_min_std,card_AMT_RECIVABLE_min_min,card_AMT_RECIVABLE_min_max,card_AMT_PAYMENT_TOTAL_CURRENT_std_mean,card_AMT_PAYMENT_TOTAL_CURRENT_std_std,card_AMT_PAYMENT_TOTAL_CURRENT_std_min,card_AMT_PAYMENT_TOTAL_CURRENT_std_max,card_SK_DPD_DEF_max_mean,card_SK_DPD_DEF_max_std,card_SK_DPD_DEF_max_min,card_SK_DPD_DEF_max_max,card_SK_DPD_std_mean,card_SK_DPD_std_std,card_SK_DPD_std_min,card_SK_DPD_std_max,card_CNT_DRAWINGS_POS_CURRENT_max_mean,card_CNT_DRAWINGS_POS_CURRENT_max_std,card_CNT_DRAWINGS_POS_CURRENT_max_min,card_CNT_DRAWINGS_POS_CURRENT_max_max,card_AMT_RECEIVABLE_PRINCIPAL_min_mean,card_AMT_RECEIVABLE_PRINCIPAL_min_std,card_AMT_RECEIVABLE_PRINCIPAL_min_min,card_AMT_RECEIVABLE_PRINCIPAL_min_max,card_NAME_CONTRACT_STATUS_Demand_std_mean,card_NAME_CONTRACT_STATUS_Demand_std_std,card_NAME_CONTRACT_STATUS_Demand_std_min,card_NAME_CONTRACT_STATUS_Demand_std_max,card_NAME_CONTRACT_STATUS_Signed_max_mean,card_NAME_CONTRACT_STATUS_Signed_max_std,card_NAME_CONTRACT_STATUS_Signed_max_min,card_NAME_CONTRACT_STATUS_Signed_max_max,card_CNT_INSTALMENT_MATURE_CUM_max_mean,card_CNT_INSTALMENT_MATURE_CUM_max_std,card_CNT_INSTALMENT_MATURE_CUM_max_min,card_CNT_INSTALMENT_MATURE_CUM_max_max,card_AMT_CREDIT_LIMIT_ACTUAL_min_mean,card_AMT_CREDIT_LIMIT_ACTUAL_min_std,card_AMT_CREDIT_LIMIT_ACTUAL_min_min,card_AMT_CREDIT_LIMIT_ACTUAL_min_max,card_AMT_DRAWINGS_ATM_CURRENT_std_mean,card_AMT_DRAWINGS_ATM_CURRENT_std_std,card_AMT_DRAWINGS_ATM_CURRENT_std_min,card_AMT_DRAWINGS_ATM_CURRENT_std_max,card_AMT_TOTAL_RECEIVABLE_mean_mean,card_AMT_TOTAL_RECEIVABLE_mean_std,card_AMT_TOTAL_RECEIVABLE_mean_min,card_AMT_TOTAL_RECEIVABLE_mean_max,card_NAME_CONTRACT_STATUS_Signed_mean_mean,card_NAME_CONTRACT_STATUS_Signed_mean_std,card_NAME_CONTRACT_STATUS_Signed_mean_min,card_NAME_CONTRACT_STATUS_Signed_mean_max,card_AMT_PAYMENT_TOTAL_CURRENT_max_mean,card_AMT_PAYMENT_TOTAL_CURRENT_max_std,card_AMT_PAYMENT_TOTAL_CURRENT_max_min,card_AMT_PAYMENT_TOTAL_CURRENT_max_max,card_AMT_RECIVABLE_std_mean,card_AMT_RECIVABLE_std_std,card_AMT_RECIVABLE_std_min,card_AMT_RECIVABLE_std_max,card_NAME_CONTRACT_STATUS_Refused_max_mean,card_NAME_CONTRACT_STATUS_Refused_max_std,card_NAME_CONTRACT_STATUS_Refused_max_min,card_NAME_CONTRACT_STATUS_Refused_max_max,card_NAME_CONTRACT_STATUS_Completed_max_mean,card_NAME_CONTRACT_STATUS_Completed_max_std,card_NAME_CONTRACT_STATUS_Completed_max_min,card_NAME_CONTRACT_STATUS_Completed_max_max,card_AMT_DRAWINGS_ATM_CURRENT_mean_mean,card_AMT_DRAWINGS_ATM_CURRENT_mean_std,card_AMT_DRAWINGS_ATM_CURRENT_mean_min,card_AMT_DRAWINGS_ATM_CURRENT_mean_max,card_AMT_TOTAL_RECEIVABLE_std_mean,card_AMT_TOTAL_RECEIVABLE_std_std,card_AMT_TOTAL_RECEIVABLE_std_min,card_AMT_TOTAL_RECEIVABLE_std_max,card_AMT_INST_MIN_REGULARITY_min_mean,card_AMT_INST_MIN_REGULARITY_min_std,card_AMT_INST_MIN_REGULARITY_min_min,card_AMT_INST_MIN_REGULARITY_min_max,card_NAME_CONTRACT_STATUS_Refused_std_mean,card_NAME_CONTRACT_STATUS_Refused_std_std,card_NAME_CONTRACT_STATUS_Refused_std_min,card_NAME_CONTRACT_STATUS_Refused_std_max,card_NAME_CONTRACT_STATUS_Signed_std_mean,card_NAME_CONTRACT_STATUS_Signed_std_std,card_NAME_CONTRACT_STATUS_Signed_std_min,card_NAME_CONTRACT_STATUS_Signed_std_max,card_SK_DPD_DEF_min_mean,card_SK_DPD_DEF_min_std,card_SK_DPD_DEF_min_min,card_SK_DPD_DEF_min_max,card_AMT_DRAWINGS_ATM_CURRENT_max_mean,card_AMT_DRAWINGS_ATM_CURRENT_max_std,card_AMT_DRAWINGS_ATM_CURRENT_max_min,card_AMT_DRAWINGS_ATM_CURRENT_max_max,card_CNT_DRAWINGS_OTHER_CURRENT_min_mean,card_CNT_DRAWINGS_OTHER_CURRENT_min_std,card_CNT_DRAWINGS_OTHER_CURRENT_min_min,card_CNT_DRAWINGS_OTHER_CURRENT_min_max,card_SK_DPD_max_mean,card_SK_DPD_max_std,card_SK_DPD_max_min,card_SK_DPD_max_max,card_AMT_DRAWINGS_OTHER_CURRENT_std_mean,card_AMT_DRAWINGS_OTHER_CURRENT_std_std,card_AMT_DRAWINGS_OTHER_CURRENT_std_min,card_AMT_DRAWINGS_OTHER_CURRENT_std_max,card_AMT_RECIVABLE_mean_mean,card_AMT_RECIVABLE_mean_std,card_AMT_RECIVABLE_mean_min,card_AMT_RECIVABLE_mean

In [ ]:
# clear memory
del credit_card_balance_df

### 4.3.4. PREVIOUS APP

In [ ]:
# check card data
previous_application_df.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15.0,Y,1.0,0.0,0.182832,0.867336,XAP,Approved,-73.0,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,35.0,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11.0,Y,1.0,NaN,NaN,NaN,XNA,Approved,-164.0,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,-1.0,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11.0,Y,1.0,NaN,NaN,NaN,XNA,Approved,-301.0,Cash through the bank,XAP,"Spouse, partner",Repeater,XNA,Cash,x-sell,Credit and cash offices,-1.0,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7.0,Y,1.0,NaN,NaN,NaN,XNA,Approved,-512.0,Cash through the bank,XAP,NaN,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1.0,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9.0,Y,1.0,NaN,NaN,NaN,Repairs,Refused,-781.0,Cash through the bank,HC,NaN,Repeater,XNA,Cash,walk-in,Credit and cash offices,-1.0,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
### FEATURE ENGINEERING

# amount ratios
previous_application_df["AMT_GIVEN_RATIO_1"]  = previous_application_df["AMT_CREDIT"] / previous_application_df["AMT_APPLICATION"]
previous_application_df["AMT_GIVEN_RATIO_2"]  = previous_application_df["AMT_GOODS_PRICE"] / previous_application_df["AMT_APPLICATION"]
previous_application_df["DOWN_PAYMENT_RATIO"] = previous_application_df["AMT_DOWN_PAYMENT"] / previous_application_df["AMT_APPLICATION"]

# logarithms
log_vars = ["AMT_CREDIT", "AMT_ANNUITY", "AMT_APPLICATION", "AMT_DOWN_PAYMENT", "AMT_GOODS_PRICE"]
previous_application_df = create_logs(previous_application_df, log_vars, replace = True)

# convert days
day_vars = ["DAYS_FIRST_DRAWING", "DAYS_FIRST_DUE", "DAYS_LAST_DUE_1ST_VERSION",
            "DAYS_LAST_DUE", "DAYS_TERMINATION", "DAYS_DECISION"]
previous_application_df = convert_days(previous_application_df, day_vars, t = 1, rounding = False, replace = True)

# number of applications
cnt_prev = previous_application_df[["SK_ID_CURR", "SK_ID_PREV"]].groupby(["SK_ID_CURR"], as_index = False).count()
cnt_prev.columns = ["SK_ID_CURR", "CNT_PREV_APPLICATIONS"]
previous_application_df = previous_application_df.merge(cnt_prev, how = "left", on = "SK_ID_CURR")

# number of contracts
cnt_prev = previous_application_df[["SK_ID_CURR", "FLAG_LAST_APPL_PER_CONTRACT"]]
cnt_prev.columns = ["SK_ID_CURR", "CNT_PREV_CONTRACTS"]
cnt_prev = cnt_prev[cnt_prev["CNT_PREV_CONTRACTS"] == "Y"]
cnt_prev = cnt_prev[["SK_ID_CURR", "CNT_PREV_CONTRACTS"]].groupby(["SK_ID_CURR"], as_index = False).count()
previous_application_df = previous_application_df.merge(cnt_prev, how = "left", on = "SK_ID_CURR")

# number ratio
previous_application_df["APPL_PER_CONTRACT_RATIO"] = previous_application_df["CNT_PREV_APPLICATIONS"] / previous_application_df["CNT_PREV_CONTRACTS"]

# loan decision ratios
previous_application_df = compute_accept_reject_ratio(previous_application_df, lags = [1, 3, 5])

# day differences
previous_application_df["DAYS_DUE_DIFF_1"] = previous_application_df["DAYS_LAST_DUE_1ST_VERSION"] - previous_application_df["DAYS_FIRST_DUE"]
previous_application_df["DAYS_DUE_DIFF_2"] = previous_application_df["DAYS_LAST_DUE"] - previous_application_df["DAYS_FIRST_DUE"]
previous_application_df["DAYS_TERMINATION_DIFF_1"] = previous_application_df["DAYS_TERMINATION"] - previous_application_df["DAYS_FIRST_DRAWING"]
previous_application_df["DAYS_TERMINATION_DIFF_2"] = previous_application_df["DAYS_TERMINATION"] - previous_application_df["DAYS_FIRST_DUE"]
previous_application_df["DAYS_TERMINATION_DIFF_3"] = previous_application_df["DAYS_TERMINATION"] - previous_application_df["DAYS_LAST_DUE"]

# application dates
previous_application_df["DAY_APPR_PROCESS_START"] = "Working day"
previous_application_df["DAY_APPR_PROCESS_START"][(previous_application_df["WEEKDAY_APPR_PROCESS_START"] == "SATURDAY") |
                               (previous_application_df["WEEKDAY_APPR_PROCESS_START"] == "SUNDAY")] = "Weekend"


##### FEATURE REMOVAL
drops = ["NAME_CLIENT_TYPE", "SK_ID_PREV"]
previous_application_df = previous_application_df.drop(columns = drops)

In [ ]:
# dummy encodnig for factors
previous_application_df = pd.get_dummies(previous_application_df, drop_first = True)

In [ ]:
# count missings
nas = count_missings(previous_application_df)
nas.head()

,Total,Percent
RATE_INTEREST_PRIVILEGED,769995,99.646318
RATE_INTEREST_PRIMARY,769995,99.646318
DAYS_TERMINATION_DIFF_1,768923,99.507589
DAYS_FIRST_DRAWING,743898,96.269062
DAYS_DUE_DIFF_1,456572,59.085733


In [ ]:
### AGGREGATIONS

# aggregate data
agg_prev = aggregate_data(previous_application_df, id_var = "SK_ID_CURR", label = "prev")

# clean up
omits = ["APPROVE_RATIO_1", "APPROVE_RATIO_3", "APPROVE_RATIO_5",
         "REJECT_RATIO_1", "REJECT_RATIO_3",  "REJECT_RATIO_5",
         "FLAG_LAST_APPL_PER_CONTRACT_Y", "CNT_PREV_CONTRACTS", "CNT_PREV_APPLICATIONS",
         "APPL_PER_CONTRACT_RATIO"]
for var in omits:
    del agg_prev["prev_" + str(var) + "_std"]
    del agg_prev["prev_" + str(var) + "_min"]
    del agg_prev["prev_" + str(var) + "_max"]

- Preparing the dataset...
- Extracted 0 factors and 160 numerics...
- Aggregating numeric features...
- Final dimensions: (278720, 640)


In [ ]:
# count missings
nas = count_missings(agg_prev)
nas.head()

,Total,Percent
prev_DAYS_TERMINATION_DIFF_1_std,278713,99.997489
prev_RATE_INTEREST_PRIVILEGED_std,278671,99.982420
prev_RATE_INTEREST_PRIMARY_std,278671,99.982420
prev_DAYS_FIRST_DRAWING_std,278668,99.981343
prev_RATE_INTEREST_PRIVILEGED_min,276036,99.037026


In [ ]:
# check data
agg_prev.head()

,prev_RATE_INTEREST_PRIMARY_mean,prev_RATE_INTEREST_PRIMARY_std,prev_RATE_INTEREST_PRIMARY_min,prev_RATE_INTEREST_PRIMARY_max,prev_NAME_PORTFOLIO_XNA_mean,prev_NAME_PORTFOLIO_XNA_std,prev_NAME_PORTFOLIO_XNA_min,prev_NAME_PORTFOLIO_XNA_max,prev_HOUR_APPR_PROCESS_START_mean,prev_HOUR_APPR_PROCESS_START_std,prev_HOUR_APPR_PROCESS_START_min,prev_HOUR_APPR_PROCESS_START_max,prev_NAME_GOODS_CATEGORY_Gardening_mean,prev_NAME_GOODS_CATEGORY_Gardening_std,prev_NAME_GOODS_CATEGORY_Gardening_min,prev_NAME_GOODS_CATEGORY_Gardening_max,prev_NAME_SELLER_INDUSTRY_Connectivity_mean,prev_NAME_SELLER_INDUSTRY_Connectivity_std,prev_NAME_SELLER_INDUSTRY_Connectivity_min,prev_NAME_SELLER_INDUSTRY_Connectivity_max,prev_NAME_CASH_LOAN_PURPOSE_Payments on other loans_mean,prev_NAME_CASH_LOAN_PURPOSE_Payments on other loans_std,prev_NAME_CASH_LOAN_PURPOSE_Payments on other loans_min,prev_NAME_CASH_LOAN_PURPOSE_Payments on other loans_max,prev_AMT_APPLICATION_mean,prev_AMT_APPLICATION_std,prev_AMT_APPLICATION_min,prev_AMT_APPLICATION_max,prev_NAME_SELLER_INDUSTRY_Industry_mean,prev_NAME_SELLER_INDUSTRY_Industry_std,prev_NAME_SELLER_INDUSTRY_Industry_min,prev_NAME_SELLER_INDUSTRY_Industry_max,prev_AMT_ANNUITY_mean,prev_AMT_ANNUITY_std,prev_AMT_ANNUITY_min,prev_AMT_ANNUITY_max,prev_WEEKDAY_APPR_PROCESS_START_MONDAY_mean,prev_WEEKDAY_APPR_PROCESS_START_MONDAY_std,prev_WEEKDAY_APPR_PROCESS_START_MONDAY_min,prev_WEEKDAY_APPR_PROCESS_START_MONDAY_max,prev_PRODUCT_COMBINATION_Card X-Sell_mean,prev_PRODUCT_COMBINATION_Card X-Sell_std,prev_PRODUCT_COMBINATION_Card X-Sell_min,prev_PRODUCT_COMBINATION_Card X-Sell_max,prev_NAME_GOODS_CATEGORY_Sport and Leisure_mean,prev_NAME_GOODS_CATEGORY_Sport and Leisure_std,prev_NAME_GOODS_CATEGORY_Sport and Leisure_min,prev_NAME_GOODS_CATEGORY_Sport and Leisure_max,prev_NAME_GOODS_CATEGORY_Fitness_mean,prev_NAME_GOODS_CATEGORY_Fitness_std,prev_NAME_GOODS_CATEGORY_Fitness_min,prev_NAME_GOODS_CATEGORY_Fitness_max,prev_NAME_CASH_LOAN_PURPOSE_XAP_mean,prev_NAME_CASH_LOAN_PURPOSE_XAP_std,prev_NAME_CASH_LOAN_PURPOSE_XAP_min,prev_NAME_CASH_LOAN_PURPOSE_XAP_max,prev_NAME_GOODS_CATEGORY_Insurance_mean,prev_NAME_GOODS_CATEGORY_Insurance_std,prev_NAME_GOODS_CATEGORY_Insurance_min,prev_NAME_GOODS_CATEGORY_Insurance_max,prev_NAME_GOODS_CATEGORY_Office Appliances_mean,prev_NAME_GOODS_CATEGORY_Office Appliances_std,prev_NAME_GOODS_CATEGORY_Office Appliances_min,prev_NAME_GOODS_CATEGORY_Office Appliances_max,prev_NAME_TYPE_SUITE_Group of people_mean,prev_NAME_TYPE_SUITE_Group of people_std,prev_NAME_TYPE_SUITE_Group of people_min,prev_NAME_TYPE_SUITE_Group of people_max,prev_NAME_CONTRACT_TYPE_Consumer loans_mean,prev_NAME_CONTRACT_TYPE_Consumer loans_std,prev_NAME_CONTRACT_TYPE_Consumer loans_min,prev_NAME_CONTRACT_TYPE_Consumer loans_max,prev_CODE_REJECT_REASON_SCOFR_mean,prev_CODE_REJECT_REASON_SCOFR_std,prev_CODE_REJECT_REASON_SCOFR_min,prev_CODE_REJECT_REASON_SCOFR_max,prev_NAME_CASH_LOAN_PURPOSE_Everyday expenses_mean,prev_NAME_CASH_LOAN_PURPOSE_Everyday expenses_std,prev_NAME_CASH_LOAN_PURPOSE_Everyday expenses_min,prev_NAME_CASH_LOAN_PURPOSE_Everyday expenses_max,prev_NAME_GOODS_CATEGORY_Weapon_mean,prev_NAME_GOODS_CATEGORY_Weapon_std,prev_NAME_GOODS_CATEGORY_Weapon_min,prev_NAME_GOODS_CATEGORY_Weapon_max,prev_PRODUCT_COMBINATION_Cash_mean,prev_PRODUCT_COMBINATION_Cash_std,prev_PRODUCT_COMBINATION_Cash_min,prev_PRODUCT_COMBINATION_Cash_max,prev_AMT_GOODS_PRICE_mean,prev_AMT_GOODS_PRICE_std,prev_AMT_GOODS_PRICE_min,prev_AMT_GOODS_PRICE_max,prev_NAME_CASH_LOAN_PURPOSE_Buying a home_mean,prev_NAME_CASH_LOAN_PURPOSE_Buying a home_std,prev_NAME_CASH_LOAN_PURPOSE_Buying a home_min,prev_NAME_CASH_LOAN_PURPOSE_Buying a home_max,prev_NAME_CASH_LOAN_PURPOSE_Buying a new car_mean,prev_NAME_CASH_LOAN_PURPOSE_Buying a new car_std,prev_NAME_CASH_LOAN_PURPOSE_Buying a new car_min,prev_NAME_CASH_LOAN_PURPOSE_Buying a new car_max,prev_NAME_GOODS_CATEGORY_Other_mean,prev_NAME_GOODS_CATEGORY_Other_std,prev_NAME_GOODS_CATEGORY_Other_min,prev_NAME_GOODS_

In [ ]:
# clear memory
del previous_application_df

# 5. DATA EXPORT

In [ ]:
# merge data
print(appl.shape)
appl = appl.merge(right = agg_buro.reset_index(), how = "left", on = "SK_ID_CURR")
print(appl.shape)
del agg_buro
appl = appl.merge(right = agg_prev.reset_index(), how = "left", on = "SK_ID_CURR")
print(appl.shape)
del agg_prev
appl = appl.merge(right = agg_inst.reset_index(), how = "left", on = "SK_ID_CURR")
print(appl.shape)
del agg_inst
appl = appl.merge(right = agg_poca.reset_index(), how = "left", on = "SK_ID_CURR")
print(appl.shape)
del agg_poca
appl = appl.merge(right = agg_card.reset_index(), how = "left", on = "SK_ID_CURR")
print(appl.shape)
del agg_card

(85656, 109)
(85656, 339)
(85656, 949)
(85656, 1097)
(85656, 1309)
(85656, 1713)


In [ ]:
##### CROSS-TABLE FEATURE ENGINEERING

# credit ratios
appl["mix_AMT_PREV_ANNUITY_RATIO"]     = appl["app_AMT_ANNUITY"] / appl["prev_AMT_ANNUITY_mean"]
appl["mix_AMT_PREV_CREDIT_RATIO"]      = appl["app_AMT_CREDIT"] / appl["prev_AMT_CREDIT_mean"]
appl["mix_AMT_PREV_GOODS_PRICE_RATIO"] = appl["app_AMT_GOODS_PRICE"] / appl["prev_AMT_GOODS_PRICE_mean"]
appl["mix_AMT_BURO_ANNUITY_RATIO"]     = appl["app_AMT_ANNUITY"] / appl["buro_AMT_ANNUITY_mean"]
appl["mix_AMT_BURO_CREDIT_RATIO"]      = appl["app_AMT_CREDIT"] / appl["buro_AMT_CREDIT_SUM_mean"]

In [ ]:
# dummy encodnig for factors
appl = pd.get_dummies(appl, drop_first = True)

In [ ]:
# label encoder for factors
#data_factors = [f for f in appl.columns if appl[f].dtype == "object"]
#for var in data_factors:
#    appl[var], _ = pd.factorize(appl[var])

In [ ]:
# count missings
nas = count_missings(appl)
nas.head()

,Total,Percent
prev_DAYS_TERMINATION_DIFF_1_std,85654,99.997665
prev_DAYS_FIRST_DRAWING_std,85642,99.983656
prev_RATE_INTEREST_PRIMARY_std,85639,99.980153
prev_RATE_INTEREST_PRIVILEGED_std,85639,99.980153
card_AMT_DRAWINGS_ATM_CURRENT_std_std,85551,99.877417


In [ ]:
# partitioning
train_df = appl[appl["SK_ID_CURR"].isin(y["SK_ID_CURR"]) == True]
test_df  = appl[appl["SK_ID_CURR"].isin(y["SK_ID_CURR"]) == False]
del appl

In [ ]:
# check dimensions
print(train_df.shape)
print(test_df.shape)

(36912, 1792)
(48744, 1792)


In [ ]:
train_df.to_csv("train_features.csv", index = False, float_format = "%.8f")
test_df.to_csv("test_features.csv",   index = False, float_format = "%.8f")
y.to_csv("train_labels.csv",         index = False, float_format = "%.8f")

### Isi dari `train_df`

In [ ]:
display(train_df.head())

SK_ID_CURR  app_CNT_CHILDREN  app_AMT_INCOME_TOTAL  app_AMT_CREDIT  \
0      100002                 0             12.218500       12.915581   
1      100003                 0             12.506181       14.072865   
2      100004                 0             11.119898       11.813037   
3      100006                 0             11.813037       12.652947   
4      100007                 0             11.707678       13.148033   

   app_AMT_ANNUITY  app_AMT_GOODS_PRICE  app_REGION_POPULATION_RELATIVE  \
0        10.114619            12.768544                        0.018801   
1        10.482892            13.937287                        0.003541   
2         8.817446            11.813037                        0.010032   
3        10.298481            12.601491                        0.008019   
4         9.992711            13.148033                        0.028663   

   app_DAYS_BIRTH  app_DAYS_EMPLOYED  app_DAYS_REGISTRATION  \
0           315.0               21.0                  122.0   
1           559.0               40.0                   40.0   
2           635.0                8.0                  142.0   
3           634.0              101.0                  328.0   
4           664.0              101.0                  144.0   

   app_DAYS_ID_PUBLISH  app_OWN_CAR_AGE  app_FLAG_MOBIL  app_FLAG_EMP_PHONE  \
0                 71.0              NaN             1.0                 1.0   
1                 10.0              NaN             1.0                 1.0   
2                 84.0             26.0             1.0                 1.0   
3                 81.0              NaN             1.0                 1.0   
4                115.0              NaN             1.0                 1.0   

   app_FLAG_WORK_PHONE  app_FLAG_CONT_MOBILE  app_FLAG_PHONE  app_FLAG_EMAIL  \
0                  0.0                   1.0             1.0             0.0   
1                  0.0                   1.0             1.0             0.0   
2                  1.0                   1.0             1.0             0.0   
3                  0.0                   1.0             0.0             0.0   
4                  0.0                   1.0             0.0             0.0   

   app_CNT_FAM_MEMBERS  app_REGION_RATING_CLIENT  \
0                  1.0                       2.0   
1                  2.0                       1.0   
2                  1.0                       2.0   
3                  2.0                       2.0   
4                  1.0                       2.0   

   app_REGION_RATING_CLIENT_W_CITY  app_HOUR_APPR_PROCESS_START  \
0                              2.0                         10.0   
1                              1.0                         11.0   
2                              2.0                          9.0   
3                              2.0                         17.0   
4                              2.0                         11.0   

   app_REG_REGION_NOT_LIVE_REGION  app_REG_REGION_NOT_WORK_REGION  \
0                             0.0                             0.0   
1                             0.0                             0.0   
2                             0.0                             0.0   
3                             0.0                             0.0   
4                             0.0                             0.0   

   app_LIVE_REGION_NOT_WORK_REGION  app_REG_CITY_NOT_LIVE_CITY  \
0                              0.0                         0.0   
1                              0.0                         0.0   
2                              0.0                         0.0   
3                              0.0                         0.0   
4                              0.0                         0.0   

   app_REG_CITY_NOT_WORK_CITY  app_LIVE_CITY_NOT_WORK_CITY  app_EXT_SOURCE_1  \
0                         0.0                          0.0          0.083037   
1                         0.0                          0.0          0.311267   
2         

### Isi dari `test_df`

In [ ]:
display(test_df.head())

SK_ID_CURR  app_CNT_CHILDREN  app_AMT_INCOME_TOTAL  app_AMT_CREDIT  \
36912      100001                 0             11.813037       13.251286   
36913      100005                 0             11.502885       12.313891   
36914      100013                 0             12.218500       13.404930   
36915      100028                 2             12.660331       14.269766   
36916      100038                 1             12.100718       13.346308   

       app_AMT_ANNUITY  app_AMT_GOODS_PRICE  app_REGION_POPULATION_RELATIVE  \
36912         9.931176            13.017005                        0.018850   
36913         9.762557            12.100718                        0.035792   
36914        11.153074            13.353477                        0.019101   
36915        10.799973            14.269766                        0.026392   
36916        10.375614            13.346308                        0.010032   

       app_DAYS_BIRTH  app_DAYS_EMPLOYED  app_DAYS_REGISTRATION  \
36912           641.0               78.0                  172.0   
36913           602.0              149.0                  304.0   
36914           668.0              149.0                   72.0   
36915           466.0               62.0                   67.0   
36916           435.0               73.0                  133.0   

       app_DAYS_ID_PUBLISH  app_OWN_CAR_AGE  app_FLAG_MOBIL  \
36912                 27.0              NaN             1.0   
36913                 54.0              NaN             1.0   
36914                117.0              5.0             1.0   
36915                140.0              NaN             1.0   
36916                142.0             16.0             1.0   

       app_FLAG_EMP_PHONE  app_FLAG_WORK_PHONE  app_FLAG_CONT_MOBILE  \
36912                 1.0                  0.0                   1.0   
36913                 1.0                  0.0                   1.0   
36914                 1.0                  0.0                   1.0   
36915                 1.0                  0.0                   1.0   
36916                 1.0                  1.0                   1.0   

       app_FLAG_PHONE  app_FLAG_EMAIL  app_CNT_FAM_MEMBERS  \
36912             0.0             1.0                  2.0   
36913             0.0             0.0                  2.0   
36914             0.0             0.0                  2.0   
36915             1.0             0.0                  4.0   
36916             0.0             0.0                  3.0   

       app_REGION_RATING_CLIENT  app_REGION_RATING_CLIENT_W_CITY  \
36912                       2.0                              2.0   
36913                       2.0                              2.0   
36914                       2.0                              2.0   
36915                       2.0                              2.0   
36916                       2.0                              2.0   

       app_HOUR_APPR_PROCESS_START  app_REG_REGION_NOT_LIVE_REGION  \
36912                         18.0                             0.0   
36913                          9.0                             0.0   
36914                         14.0                             0.0   
36915                         11.0                             0.0   
36916                          5.0                             0.0   

       app_REG_REGION_NOT_WORK_REGION  app_LIVE_REGION_NOT_WORK_REGION  \
36912                             0.0                              0.0   
36913                             0.0                              0.0   
36914                             0.0                              0.0   
36915                             0.0                              0.0   
36916                             0.0                              0.0   

       app_REG_CITY_NOT_LIVE_CITY  app_REG_CITY_NOT_WORK_CITY  \
36912                         0.0                         0.0   
36913                         0.0                         0.0   
36914      

### Isi dari `y`

In [ ]:
display(y.head())

,SK_ID_CURR,TARGET
0,100002,1.0
1,100003,0.0
2,100004,0.0
3,100006,0.0
4,100007,0.0
